In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import pandas as pd
import numpy as np

In [8]:
data = pd.read_csv('/content/drive/MyDrive/Ybigta/keyword_labeled_whole_data.csv')
data = data.drop('Unnamed: 0', axis = 1)
data.head()

,row_num,location,cafe_name,review,LSTM_pp,label_org,label_mod,LDA_pp,keyword
0,0,강남역,카페노티드 청담,그냥 그렇다 생각했는데 다른 곳에서 도넛 먹고나면? ㄷ ㄷ 킹티드 .. 잘되는 이유...,"그냥/Noun,그렇다/Adjective,생각/Noun,하다/Verb,다른/Noun,...",4,1,그냥 그렇다 생각 다른 곳 넛 킹 드 이유 있다 그냥 갤러리아 지하 게 갑자기 확장...,커피&디저트
1,1,강남역,카페노티드 청담,우유크림 개맛있슴 근데 그만큼 혼잡해서 또 가고싶지 않은 곳..,"우유/Noun,크림/Noun,개/Noun,맛있다/Adjective,근데/Adverb...",3,3,우유 크림 개 맛있다 혼잡하다 또 곳,커피&디저트
2,2,강남역,카페노티드 청담,도넛은 맛있는데 종업원들 대응이 너무 느려요. 한남이랑 삼성점도 가봤는데 여기가 유...,"도넛/Noun,맛있다/Adjective,종업원/Noun,대응/Noun,너무/Adve...",2,0,도넛 맛있다 종업원 대응 느리다 한남 삼성 점도 여기 손 느리다 듯 직원 많다 왜 ...,서비스
3,3,강남역,카페노티드 청담,도넛이 맛은 있는데 줄이 진짜 너엉어ㅓㅓ어어어어무 길다 예약해서 가져갈거면 가고 가...,"도넛/Noun,맛/Noun,있다/Adjective,줄이다/Verb,진짜/Noun,너...",4,1,도넛 맛 있다 진짜 엉 어어 무길 예약 거 절대 못,커피&디저트
4,4,강남역,카페노티드 청담,감사합니다,감사하다/Verb,1,0,NaN,기타


##필요한 라이브러리 import

In [6]:
!pip install konlpy 
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
!pip install git+https://github.com/ssut/py-hanspell.git

from hanspell import spell_checker
def grammar_check(text):
  spelled_sent = spell_checker.check(text)
  hanspell_sent = spelled_sent.checked
  return hanspell_sent

!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

from pykospacing import spacing

     |████████████████████████████████| 19.4MB 1.5MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
     |████████████████████████████████| 460kB 36.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-jsss1aih
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-jsss1aih
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp37-none-any.whl size=4854 sha256=18985739c6848ae7513b58715d035e6122a2d1b777e25d64afd379bb60fea9bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-vrb_s1b4/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-obh62jog
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-obh62jog
   

##감성분석 함수

In [9]:
tokenizer = Tokenizer(5542)
okt = Okt()
max_len=30

In [10]:
import json

with open('/content/drive/MyDrive/Ybigta/신입기수 프로젝트/감성분석 모델링 추가(Bi-LSTM, Attention)/wordIndex(BiLSTM2).json') as json_file:
  word_index = json.load(json_file)
  tokenizer.word_index = word_index

from keras.models import load_model
model = load_model('/content/drive/MyDrive/Ybigta/신입기수 프로젝트/감성분석 모델링 추가(Bi-LSTM, Attention)/BiLSTM_model(sigmoid).h5')

In [11]:
import re

punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&' 
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2",
                 "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 
                 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi'} 

def clean_punc(text, punct, mapping): 
  for p in mapping: 
    text = text.replace(p, mapping[p]) 
  for p in punct: 
    text = text.replace(p, f' {p} ') 
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''} 
  for s in specials: 
    text = text.replace(s, specials[s]) 
   
  return text.strip() 

def clean_text(texts): 
  corpus = [] 
  for i in range(0, len(texts)): 
    review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation 
    review = re.sub(r'\d+','', str(texts[i]))# remove number 
    review = review.lower() #lower case 
    review = re.sub(r'\s+', ' ', review) #remove extra space 
    review = re.sub(r'<[^>]+>','',review) #remove Html tags 
    review = re.sub(r'\s+', ' ', review) #remove spaces 
    review = re.sub(r"^\s+", '', review) #remove space from start 
    review = re.sub(r'\s+$', '', review) #remove space from the end 
    review = re.sub(r'♥','',review)
    review = re.sub(r'~','',review)
    review = re.sub(r'&','',review)
    review = re.sub(r'😱','',review)
    corpus.append(review) 
  return corpus

# removing emoji

def rmEmoji(text):
  review = text.encode('utf-8', 'ignore').decode('utf-8')
  return review

def rmEmoji_ascii(inputString):
  review = example.encode('ascii','ignore').decode('ascii')
  return review

In [12]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
# NLTK Data 다운
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = open('/content/drive/MyDrive/Ybigta/korean_stopwords.txt').read()


# 위의 불용어는 명사가 아닌 단어 중에서 저자가 임의로 선정한 것으로 실제 의미있는 선정 기준이 아님
stop_words=stop_words.split('\n')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
def tokenize_tagged(text):

  temp_X = okt.pos(text, norm=True, stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stop_words] # 불용어 제거
  return ['/'.join(t) for t in temp_X]

In [33]:
def sentiment_predict(new_sentence):
  text1 = clean_punc(new_sentence, punct, punct_mapping)
  text2 = ''.join(clean_text(text1))
  text3 = spacing(text2)
  text4 = grammar_check(text3)
  text_ = tokenize_tagged(text4)
  encoded = tokenizer.texts_to_sequences([text_]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(model.predict(pad_new)) * 100# 예측
  score = round(score, 2)
  
  return score

In [19]:
sentiment_predict('여기는 커피가 정말 너무 맛있어요 최고예요 직원도 친절해요')

99.87


99.87

##데이터프레임에 감성분석 칼럼 추가

In [20]:
data['sentiment_score'] = data['review'].apply(lambda x: sentiment_predict(x))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2.95
93.59
56.89
99.86
99.58
89.46
56.89
99.31
16.84
1.62
6.4
96.53
46.65
3.07
91.13
99.39
99.73
13.37
51.22
98.85
4.81
4.51
2.4
7.51
2.27
6.26
27.95
56.89
99.77
9.54
0.63
3.26
75.94
98.96
6.42
99.83
99.83
73.89
4.0
1.61
9.16
99.83
96.5
56.89
89.91
98.61
88.66
99.56
41.76
69.61
6.92
2.48
94.91
9.94
0.89
2.24
92.31
2.57
64.11
3.18
99.37
6.16
56.89
76.66
21.94
14.74
9.54
6.4
2.16
3.24
30.51
1.45
98.28
8.78
1.36
56.89
99.94
82.28
70.54
20.25
96.4
56.89
97.73
12.37
99.88
99.83
96.83
98.78
99.59
50.39
99.43
56.89
99.74
26.77
99.83
98.98
99.43
99.61
12.3
99.29
68.39
2.41
98.95
99.56
99.79
87.78
99.83
98.73
99.43
99.64
99.83
99.32
66.0
98.33
9.54
38.61
99.31
96.4
99.65
98.5
96.4
99.68
6.49
99.57
96.4
9.54
34.62
36.34
99.83
1.38
92.68
2.18
22.67
73.07
6.4
2.22
97.52
39.88
13.35
11.14
92.24
62.98
8.37
98.43
8.25
46.14
1.37
3.46
1.97
99.55
47.34
94.44
99.41
6.4
84.73
4.54
86.87
12.63
57.55
96.4
99.91
96.82
99.5
99.06
99.3
3.18
98.95
6.4
98.68
99.81
99.9
91.3
9

In [46]:
data

,row_num,location,cafe_name,review,LSTM_pp,label_org,label_mod,LDA_pp,keyword,sentiment_score,final_label
0,0,강남역,카페노티드 청담,그냥 그렇다 생각했는데 다른 곳에서 도넛 먹고나면? ㄷ ㄷ 킹티드 .. 잘되는 이유...,"그냥/Noun,그렇다/Adjective,생각/Noun,하다/Verb,다른/Noun,...",4,1,그냥 그렇다 생각 다른 곳 넛 킹 드 이유 있다 그냥 갤러리아 지하 게 갑자기 확장...,커피&디저트,46.49,4
1,1,강남역,카페노티드 청담,우유크림 개맛있슴 근데 그만큼 혼잡해서 또 가고싶지 않은 곳..,"우유/Noun,크림/Noun,개/Noun,맛있다/Adjective,근데/Adverb...",3,3,우유 크림 개 맛있다 혼잡하다 또 곳,커피&디저트,99.28,3
2,2,강남역,카페노티드 청담,도넛은 맛있는데 종업원들 대응이 너무 느려요. 한남이랑 삼성점도 가봤는데 여기가 유...,"도넛/Noun,맛있다/Adjective,종업원/Noun,대응/Noun,너무/Adve...",2,0,도넛 맛있다 종업원 대응 느리다 한남 삼성 점도 여기 손 느리다 듯 직원 많다 왜 ...,서비스,1.61,2
3,3,강남역,카페노티드 청담,도넛이 맛은 있는데 줄이 진짜 너엉어ㅓㅓ어어어어무 길다 예약해서 가져갈거면 가고 가...,"도넛/Noun,맛/Noun,있다/Adjective,줄이다/Verb,진짜/Noun,너...",4,1,도넛 맛 있다 진짜 엉 어어 무길 예약 거 절대 못,커피&디저트,32.06,4
4,4,강남역,카페노티드 청담,감사합니다,감사하다/Verb,1,0,NaN,기타,99.68,1
...,...,...,...,...,...,...,...,...,...,...,...
34484,34484,한남동,모모,조금 아쉬워요,"조금/Noun,아쉽다/Adjective",2,0,조금 아쉽다,기타,9.54,2
34485,34485,한남동,모모,우리는 결국 모두 어떤 기대 없이 하도록 돼 있는 일을 하게 된다. 그래서 이 곳은...,"우리/Noun,결국/Adverb,모두/Noun,어떻다/Adjective,기대/Nou...",5,1,우리 모두 어떻다 기대 있다 일 곳 점 헤헤,커피&디저트,51.70,5
34486,34486,한남동,모모,가격도 합리적이고 분위기두 굿!,"가격/Noun,합리/Noun,분위기/Noun,후/Noun,굿/Noun",5,1,가격 합리 분위기 후 굿,가격,99.80,5
34487,34487,한남동,모모,조용한 골목에 있어서 일단 너무 좋아요. 분위기도 예쁘고. 여긴 독특하게 시간당으로...,"조용하다/Adjective,골목/Noun,있다/Adjective,일단/Noun,너무...",4,1,조용하다 골목 있다 일단 좋다 분위기 예쁘다 독특하다 시간 계산 계산기 편 아니다 ...,분위기,79.08,4


In [32]:
data.to_csv("/content/drive/MyDrive/Ybigta/final_dataset.csv")

In [47]:
def get_final_score(score):
  if score > 90 :
    return 5
  elif score > 60:
    return 4
  elif score > 40:
    return 3
  elif score > 20:
    return 2
  else:
    return 1

In [78]:
data['sentiment_score'][1]

99.28

In [83]:
for i in range(34489): 
  if data['label_org'][i] > 0:
    data['final_label'][i] = data['label_org'][i]
  else:
    data['final_label'][i] = get_final_score(data['sentiment_score'][i])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [84]:
data

,row_num,location,cafe_name,review,LSTM_pp,label_org,label_mod,LDA_pp,keyword,sentiment_score,final_label,final_label2
0,0,강남역,카페노티드 청담,그냥 그렇다 생각했는데 다른 곳에서 도넛 먹고나면? ㄷ ㄷ 킹티드 .. 잘되는 이유...,"그냥/Noun,그렇다/Adjective,생각/Noun,하다/Verb,다른/Noun,...",4,1,그냥 그렇다 생각 다른 곳 넛 킹 드 이유 있다 그냥 갤러리아 지하 게 갑자기 확장...,커피&디저트,46.49,4,4
1,1,강남역,카페노티드 청담,우유크림 개맛있슴 근데 그만큼 혼잡해서 또 가고싶지 않은 곳..,"우유/Noun,크림/Noun,개/Noun,맛있다/Adjective,근데/Adverb...",3,3,우유 크림 개 맛있다 혼잡하다 또 곳,커피&디저트,99.28,3,3
2,2,강남역,카페노티드 청담,도넛은 맛있는데 종업원들 대응이 너무 느려요. 한남이랑 삼성점도 가봤는데 여기가 유...,"도넛/Noun,맛있다/Adjective,종업원/Noun,대응/Noun,너무/Adve...",2,0,도넛 맛있다 종업원 대응 느리다 한남 삼성 점도 여기 손 느리다 듯 직원 많다 왜 ...,서비스,1.61,2,2
3,3,강남역,카페노티드 청담,도넛이 맛은 있는데 줄이 진짜 너엉어ㅓㅓ어어어어무 길다 예약해서 가져갈거면 가고 가...,"도넛/Noun,맛/Noun,있다/Adjective,줄이다/Verb,진짜/Noun,너...",4,1,도넛 맛 있다 진짜 엉 어어 무길 예약 거 절대 못,커피&디저트,32.06,4,4
4,4,강남역,카페노티드 청담,감사합니다,감사하다/Verb,1,0,NaN,기타,99.68,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34484,34484,한남동,모모,조금 아쉬워요,"조금/Noun,아쉽다/Adjective",2,0,조금 아쉽다,기타,9.54,2,2
34485,34485,한남동,모모,우리는 결국 모두 어떤 기대 없이 하도록 돼 있는 일을 하게 된다. 그래서 이 곳은...,"우리/Noun,결국/Adverb,모두/Noun,어떻다/Adjective,기대/Nou...",5,1,우리 모두 어떻다 기대 있다 일 곳 점 헤헤,커피&디저트,51.70,5,5
34486,34486,한남동,모모,가격도 합리적이고 분위기두 굿!,"가격/Noun,합리/Noun,분위기/Noun,후/Noun,굿/Noun",5,1,가격 합리 분위기 후 굿,가격,99.80,5,5
34487,34487,한남동,모모,조용한 골목에 있어서 일단 너무 좋아요. 분위기도 예쁘고. 여긴 독특하게 시간당으로...,"조용하다/Adjective,골목/Noun,있다/Adjective,일단/Noun,너무...",4,1,조용하다 골목 있다 일단 좋다 분위기 예쁘다 독특하다 시간 계산 계산기 편 아니다 ...,분위기,79.08,4,4


In [88]:
data

,row_num,location,cafe_name,review,LSTM_pp,label_org,label_mod,LDA_pp,keyword,sentiment_score,final_label
0,0,강남역,카페노티드 청담,그냥 그렇다 생각했는데 다른 곳에서 도넛 먹고나면? ㄷ ㄷ 킹티드 .. 잘되는 이유...,"그냥/Noun,그렇다/Adjective,생각/Noun,하다/Verb,다른/Noun,...",4,1,그냥 그렇다 생각 다른 곳 넛 킹 드 이유 있다 그냥 갤러리아 지하 게 갑자기 확장...,커피&디저트,46.49,4
1,1,강남역,카페노티드 청담,우유크림 개맛있슴 근데 그만큼 혼잡해서 또 가고싶지 않은 곳..,"우유/Noun,크림/Noun,개/Noun,맛있다/Adjective,근데/Adverb...",3,3,우유 크림 개 맛있다 혼잡하다 또 곳,커피&디저트,99.28,3
2,2,강남역,카페노티드 청담,도넛은 맛있는데 종업원들 대응이 너무 느려요. 한남이랑 삼성점도 가봤는데 여기가 유...,"도넛/Noun,맛있다/Adjective,종업원/Noun,대응/Noun,너무/Adve...",2,0,도넛 맛있다 종업원 대응 느리다 한남 삼성 점도 여기 손 느리다 듯 직원 많다 왜 ...,서비스,1.61,2
3,3,강남역,카페노티드 청담,도넛이 맛은 있는데 줄이 진짜 너엉어ㅓㅓ어어어어무 길다 예약해서 가져갈거면 가고 가...,"도넛/Noun,맛/Noun,있다/Adjective,줄이다/Verb,진짜/Noun,너...",4,1,도넛 맛 있다 진짜 엉 어어 무길 예약 거 절대 못,커피&디저트,32.06,4
4,4,강남역,카페노티드 청담,감사합니다,감사하다/Verb,1,0,NaN,기타,99.68,1
...,...,...,...,...,...,...,...,...,...,...,...
34484,34484,한남동,모모,조금 아쉬워요,"조금/Noun,아쉽다/Adjective",2,0,조금 아쉽다,기타,9.54,2
34485,34485,한남동,모모,우리는 결국 모두 어떤 기대 없이 하도록 돼 있는 일을 하게 된다. 그래서 이 곳은...,"우리/Noun,결국/Adverb,모두/Noun,어떻다/Adjective,기대/Nou...",5,1,우리 모두 어떻다 기대 있다 일 곳 점 헤헤,커피&디저트,51.70,5
34486,34486,한남동,모모,가격도 합리적이고 분위기두 굿!,"가격/Noun,합리/Noun,분위기/Noun,후/Noun,굿/Noun",5,1,가격 합리 분위기 후 굿,가격,99.80,5
34487,34487,한남동,모모,조용한 골목에 있어서 일단 너무 좋아요. 분위기도 예쁘고. 여긴 독특하게 시간당으로...,"조용하다/Adjective,골목/Noun,있다/Adjective,일단/Noun,너무...",4,1,조용하다 골목 있다 일단 좋다 분위기 예쁘다 독특하다 시간 계산 계산기 편 아니다 ...,분위기,79.08,4


In [89]:
data.to_csv("/content/drive/MyDrive/Ybigta/final_of_final_dataset.csv")

In [39]:
data = pd.read_csv('/content/drive/MyDrive/Ybigta/final_dataset.csv')
data = data.drop('Unnamed: 0', axis = 1)
data.head()

,row_num,location,cafe_name,review,LSTM_pp,label_org,label_mod,LDA_pp,keyword,sentiment_score
0,0,강남역,카페노티드 청담,그냥 그렇다 생각했는데 다른 곳에서 도넛 먹고나면? ㄷ ㄷ 킹티드 .. 잘되는 이유...,"그냥/Noun,그렇다/Adjective,생각/Noun,하다/Verb,다른/Noun,...",4,1,그냥 그렇다 생각 다른 곳 넛 킹 드 이유 있다 그냥 갤러리아 지하 게 갑자기 확장...,커피&디저트,46.49
1,1,강남역,카페노티드 청담,우유크림 개맛있슴 근데 그만큼 혼잡해서 또 가고싶지 않은 곳..,"우유/Noun,크림/Noun,개/Noun,맛있다/Adjective,근데/Adverb...",3,3,우유 크림 개 맛있다 혼잡하다 또 곳,커피&디저트,99.28
2,2,강남역,카페노티드 청담,도넛은 맛있는데 종업원들 대응이 너무 느려요. 한남이랑 삼성점도 가봤는데 여기가 유...,"도넛/Noun,맛있다/Adjective,종업원/Noun,대응/Noun,너무/Adve...",2,0,도넛 맛있다 종업원 대응 느리다 한남 삼성 점도 여기 손 느리다 듯 직원 많다 왜 ...,서비스,1.61
3,3,강남역,카페노티드 청담,도넛이 맛은 있는데 줄이 진짜 너엉어ㅓㅓ어어어어무 길다 예약해서 가져갈거면 가고 가...,"도넛/Noun,맛/Noun,있다/Adjective,줄이다/Verb,진짜/Noun,너...",4,1,도넛 맛 있다 진짜 엉 어어 무길 예약 거 절대 못,커피&디저트,32.06
4,4,강남역,카페노티드 청담,감사합니다,감사하다/Verb,1,0,NaN,기타,99.68
